# Import Libraries

In [1]:
%tensorflow_version 1.x
!pip install -q gpt-2-simple
import gpt_2_simple as gpt2
from datetime import datetime
from google.colab import files
from google.colab import drive
from collections import Counter
import os
import numpy as np
import glob
import pandas as pd
from tqdm import tqdm
!python -m spacy download en
import spacy
nlp = spacy.load('en')
spacy.prefer_gpu()

TensorFlow 1.x selected.
The TensorFlow contrib module will not be included in TensorFlow 2.0.
For more information, please see:
  * https://github.com/tensorflow/community/blob/master/rfcs/20180907-contrib-sunset.md
  * https://github.com/tensorflow/addons
  * https://github.com/tensorflow/io (for I/O related ops)
If you depend on functionality not listed there, please file an issue.

✔ Download and installation successful
You can now load the model via spacy.load('en_core_web_sm')
✔ Linking successful
/usr/local/lib/python3.6/dist-packages/en_core_web_sm -->
/usr/local/lib/python3.6/dist-packages/spacy/data/en
You can now load the model via spacy.load('en')


True

# Mount Google Drive

In [2]:
gpt2.mount_gdrive()

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


# Import and Load Trained Model

In [3]:
# Copy checkpoint
gpt2.copy_checkpoint_from_gdrive(run_name='run1')

# Start session and load model
sess = gpt2.start_tf_sess()
gpt2.load_gpt2(sess, run_name='run1', multi_gpu=True)

Loading checkpoint checkpoint/run1/model-1000
INFO:tensorflow:Restoring parameters from checkpoint/run1/model-1000


In [0]:
def create_chat_outputs(prompts_list, num_responses = 100,moving_window=3, min_len_response=15, remove_responses=False, num_samples = 1):

  # Initialize an emtpy list for the future addition of dataframes.
  df_list = []

  # Iteratively enter each prompt until num_responses are produced by the nlp model. 
  for prompt in tqdm(prompts_list):

    notebook_directory = os.getcwd()
    path = '/content/drive/My Drive/5_datascience/2_context_chatbot/ouputs'
    extension = 'csv'
    os.chdir(path)
    csv_names = glob.glob('*.{}'.format(extension))
    os.chdir(notebook_directory)
    num_complete = len(csv_names)

    print(prompt)
    output_dict = {}

    # initialize inputs and chatbot reponses
    input_text = ""
    chatbot_response = ""
    oldtext=""
    temp=.4 # what does temp do?
    moving_window_used = False
    output_list = []
    temp_list = []

    i=1
    input_text = "Client: " + prompt + "\nTHERAPIST"

    
    
    prev_output_size = 0

    # Generate text from the combined input text
    while(len(output_list) < num_responses):
      generated_texts = gpt2.generate(sess,
                  length=120,
                  temperature=temp,
                  prefix=input_text,
                  nsamples=num_responses - len(output_list),
                  batch_size=1,
                  return_as_list=True
                  )
      
       

      # Determine where to split the response single text by checking if moving window is being used.
      if moving_window_used:
        split_index = moving_window 
      else:
        split_index = i

      responses = []

      # check if the outputs meet the requirements. 
      for k in range(len(generated_texts)):
        sample_response = generated_texts[k].split("THERAPIST:")[split_index].split("CLIENT:")[0].strip()
        if len(sample_response) > 1 and len(output_list) < num_responses:
          output_list.append(sample_response)
      
      output_set = set(output_list)
      output_list = list(output_set)

      for l in range(len(output_list) - prev_output_size):
        temp_list.append(temp)

      prev_output_size = len(output_list)
      temp += .05 # randomness parameter

      print(f'For prompt number {len(df_list)}, {len(output_list)} outputs were created.')

    output_dict['input'] = prompt
    output_dict['output_list'] = output_list
    output_dict['temp_list'] = temp_list

    single_output_df = pd.DataFrame(output_dict)
    single_output_df.to_csv(f"/content/drive/My Drive/5_datascience/2_context_chatbot/ouputs/prompt{num_complete}_output.csv")
    df_list.append(single_output_df)


  return(df_list)

    

In [0]:
all_prompts = pd.read_csv('/content/drive/My Drive/5_datascience/2_context_chatbot/sample_prompts.csv')
notebook_directory = os.getcwd()
path = '/content/drive/My Drive/5_datascience/2_context_chatbot/ouputs'
extension = 'csv'
os.chdir(path)
csv_names = glob.glob('*.{}'.format(extension))
os.chdir(notebook_directory)
num_complete = len(csv_names)

In [6]:
all_outputs = create_chat_outputs(all_prompts['Prompt'][num_complete:])

  0%|          | 0/2 [00:00<?, ?it/s]

I can't get excited about anything 
For prompt number 0, 55 outputs were created.
For prompt number 0, 73 outputs were created.
For prompt number 0, 82 outputs were created.
For prompt number 0, 94 outputs were created.
For prompt number 0, 97 outputs were created.
For prompt number 0, 98 outputs were created.
For prompt number 0, 99 outputs were created.
For prompt number 0, 99 outputs were created.


 50%|█████     | 1/2 [04:45<04:45, 285.66s/it]

For prompt number 0, 100 outputs were created.
I don't feel anything 
For prompt number 1, 48 outputs were created.
For prompt number 1, 71 outputs were created.
For prompt number 1, 86 outputs were created.
For prompt number 1, 90 outputs were created.
For prompt number 1, 95 outputs were created.
For prompt number 1, 99 outputs were created.


100%|██████████| 2/2 [09:42<00:00, 291.01s/it]

For prompt number 1, 100 outputs were created.


# Merge the outputs into a single csv

In [8]:
notebook_directory = os.getcwd()
path = '/content/drive/My Drive/5_datascience/2_context_chatbot/ouputs'
extension = 'csv'
os.chdir(path)
csv_names = glob.glob('*.{}'.format(extension))
os.chdir(notebook_directory)
num_complete = len(csv_names)

outputs_list = []
for csv_name in tqdm(csv_names):
    df = pd.read_csv(path+'/'+csv_name, low_memory=False)
    outputs_list.append(df)


100%|██████████| 129/129 [00:48<00:00,  2.65it/s]


In [0]:
all_outputs = pd.concat(outputs_list)

all_outputs.to_csv('/content/drive/My Drive/5_datascience/2_context_chatbot/ouputs/all_outputs.csv', index = False)